<a href="https://colab.research.google.com/github/CHINHUICHU/Auto-GPT/blob/master/HW2_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/deankuo/ADL24-HW2.git
%cd ADL24-HW2
!pip install -e tw_rouge

Cloning into 'ADL24-HW2'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 71 (delta 22), reused 26 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 18.58 KiB | 6.19 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/ADL24-HW2
Obtaining file:///content/ADL24-HW2/tw_rouge
  Preparing metadata (setup.py) ... done
  Running setup.py develop for tw_rouge


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

from ckiptagger import WS, data_utils
from rouge import Rouge
import json

cache_dir = os.environ.get("XDG_CACHE_HOME", os.path.join(os.getenv("HOME"), ".cache"))
download_dir = os.path.join(cache_dir, "ckiptagger")
data_dir = os.path.join(cache_dir, "ckiptagger/data")
os.makedirs(download_dir, exist_ok=True)
os.environ["TF_USE_LEGACY_KERAS"] = "1"
if not os.path.exists(os.path.join(data_dir, "model_ws")):
    data_utils.download_data_url(download_dir)

ws = WS(data_dir, disable_cuda=False)


def tokenize_and_join(sentences):
    return [" ".join(toks) for toks in ws(sentences)]


rouge = Rouge()


def get_rouge(preds, refs, avg=True, ignore_empty=False):
    """wrapper around: from rouge import Rouge
    Args:
        preds: string or list of strings
        refs: string or list of strings
        avg: bool, return the average metrics if set to True
        ignore_empty: bool, ignore empty pairs if set to True
    """
    if not isinstance(preds, list):
        preds = [preds]
    if not isinstance(refs, list):
        refs = [refs]
    preds, refs = tokenize_and_join(preds), tokenize_and_join(refs)
    return rouge.get_scores(preds, refs, avg=avg, ignore_empty=ignore_empty)


refs, preds = {}, {}

reference = "/content/drive/My Drive/adl-hw2-data/public.jsonl"
submission = "/content/drive/My Drive/adl-hw2-output/predictions-2.jsonl"

with open(reference) as file:
    for line in file:
        line = json.loads(line)
        refs[line['id']] = line['title'].strip() + '\n'

with open(submission) as file:
    for line in file:
        line = json.loads(line)
        preds[line['id']] = line['title'].strip() + '\n'

keys =  refs.keys()
refs = [refs[key] for key in keys]
preds = [preds[key] for key in keys]

print(json.dumps(get_rouge(preds, refs), indent=2))




/usr/local/lib/python3.10/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


{
  "rouge-1": {
    "r": 0.25358283127640974,
    "p": 0.2817865142534032,
    "f": 0.25984384706048624
  },
  "rouge-2": {
    "r": 0.09025022719120931,
    "p": 0.1001225754563468,
    "f": 0.0921234501253248
  },
  "rouge-l": {
    "r": 0.22102576654556383,
    "p": 0.24552238978048643,
    "f": 0.2262737524065005
  }
}
